In [1]:
# Generic
import typing
import os
from pathlib import Path
import typing
import json

# Plotting
import matplotlib.pyplot as plt

# Numerical Computing
import numpy as np
import torch

# muutils
from zanj.zanj import ZANJ, ZANJ_GLOBAL_DEFAULTS

# Our Code
from muutils.nbutils.configure_notebook import configure_notebook
from maze_transformer.training.config import ConfigHolder, ZanjHookedTransformer, BaseGPTConfig, TrainConfig
from maze_dataset import MazeDataset, MazeDatasetConfig
from maze_dataset.dataset.configs import MAZE_DATASET_CONFIGS
from maze_dataset.generation import LatticeMazeGenerators
from maze_transformer.training.train_model import TrainingResult, train_model
from maze_transformer.training.wandb_logger import WandbProject


In [2]:
# set global defaults for ZANJ
ZANJ_GLOBAL_DEFAULTS.external_array_threshold = 1024
ZANJ_GLOBAL_DEFAULTS.external_list_threshold = 1024

# paths
PATH_EXAMPLES: Path = Path("../examples/")
PATH_DATA: Path = Path("../data/")

# reproducibility and device
DEVICE = configure_notebook(seed=42, dark_mode=True)
print(f"{DEVICE = }")

set up plots with PLOT_MODE = 'inline', FIG_OUTPUT_FMT = None, FIG_BASEPATH = None
DEVICE = device(type='cpu')


In [3]:
print(f"{list(MAZE_DATASET_CONFIGS.keys()) = }")

# if you want to specify a custom config, you can do so here
CFG_CUSTOM: ConfigHolder = ConfigHolder(
    name = "custom",
    dataset_cfg = MazeDatasetConfig(
		name="custom-dataset",
		grid_n=6,
		n_mazes=10000,
		maze_ctor=LatticeMazeGenerators.gen_dfs,
	),
    model_cfg = BaseGPTConfig(
        name="custom-model",
        act_fn="gelu",
        d_model=8,
        d_head=4,
        n_layers=2,
    ),
    train_cfg = TrainConfig(
        name="custom-train",
        optimizer=torch.optim.RMSprop,
        optimizer_kwargs=dict(lr=0.0001),
        batch_size=16,
        dataloader_cfg=dict(
            shuffle=True,
            num_workers=0,
            drop_last=False,
        ),
        intervals_count=dict(
            print_loss=100,
            checkpoint=5,
            eval_fast=10,
            eval_slow=5,
        )
    ),
)

CFG_HALLWAY: ConfigHolder = ConfigHolder(
    name = "hallway_v3",
    dataset_cfg = MazeDatasetConfig(
		name="hallway",
		grid_n=7,
		n_mazes=3_000_000,
		maze_ctor=LatticeMazeGenerators.gen_dfs,
        maze_ctor_kwargs=dict(
            do_forks=False,
        ),
        applied_filters=[{'name': 'collect_generation_meta', 'args': (), 'kwargs': {}}],
        seq_len_max=256,
	),
    model_cfg = BaseGPTConfig(
        name="custom-model",
        act_fn="gelu",
        d_model=128,
        d_head=32,
        n_layers=6,
    ),
    train_cfg = TrainConfig(
        name="custom-train",
        optimizer=torch.optim.AdamW,
        optimizer_kwargs=dict(lr=0.0001),
        batch_size=32,
        dataloader_cfg=dict(
            shuffle=False,
            num_workers=8,
            drop_last=False,
        ),
        intervals_count=dict(
            print_loss=100,
            checkpoint=20,
            eval_fast=100,
            eval_slow=50,
        ),
        validation_dataset_cfg=100,
    ),
)

list(MAZE_DATASET_CONFIGS.keys()) = ['test-g3-n5-a_dfs-h73257', 'demo_small-g3-n100-a_dfs-h44636', 'demo-g6-n10K-a_dfs-h50618']


In [4]:
# this is for training a "real" demo model
CFG_DEMO: ConfigHolder = ConfigHolder.get_config_multisource(
    cfg_names=("test-g3-n5-a_dfs-h73257", "tiny-v1", "sweep-v1"),
)

# this is smaller, for testing
CFG_TEST: ConfigHolder = ConfigHolder.get_config_multisource(
    cfg_names=("demo_small-g3-n100-a_dfs-h44636", "nano-v1", "test-v1"),
)

In [5]:
# here is where to specify which config to actually use
CFG: ConfigHolder = CFG_TEST

In [6]:
print(json.dumps(CFG.summary(), indent=2))

{
  "name": "multsrc_demo_small-g3-n100-a_dfs-h44636_nano-v1_test-v1",
  "dataset_cfg": {
    "name": "demo_small",
    "fname": "demo_small-g3-n100-a_dfs-h44636",
    "sdc_hash": 89724264431769658998652566433510669623512452901670271738715908684739630044636,
    "seed": 42,
    "seq_len_min": 1,
    "seq_len_max": 512,
    "applied_filters": [],
    "grid_n": 3,
    "grid_shape": [
      3,
      3
    ],
    "n_mazes": 100,
    "maze_ctor_name": "gen_dfs",
    "maze_ctor_kwargs": {}
  },
  "model_cfg": {
    "name": "nano-v1",
    "act_fn": "gelu",
    "d_model": 8,
    "d_head": 4,
    "n_layers": 2,
    "positional_embedding_type": "standard",
    "weight_processing": {
      "are_layernorms_folded": false,
      "are_weights_processed": false
    },
    "n_heads": 2
  },
  "train_cfg": {
    "name": "test-v1",
    "optimizer": "RMSprop",
    "optimizer_kwargs": {
      "lr": 0.0001
    },
    "batch_size": 16,
    "dataloader_cfg": {
      "shuffle": true,
      "num_workers": 0,
 

In [7]:
# get just the dataset, generating it if needed. 
# This step can be skipped if you set `do_generate_dataset=True` when calling `train_model`
# or if the dataset in question already exists

# load the dataset
DATASET: MazeDataset = MazeDataset.from_config(CFG.dataset_cfg, verbose=True, local_base_path=PATH_DATA)

trying to get the dataset 'demo_small-g3-n100-a_dfs-h44636'
seeing if we can download the dataset...
no download found, or download failed
generating dataset...


generating & solving mazes: 100%|██████████| 100/100 [00:00<00:00, 1562.52maze/s]

saving dataset to ..\data\demo_small-g3-n100-a_dfs-h44636.zanj
Got dataset demo_small with 100 items. output.cfg.to_fname() = 'demo_small-g3-n100-a_dfs-h91156'


In [8]:
result: TrainingResult = train_model(
	base_path=PATH_DATA,
    cfg=CFG,
	wandb_project=WandbProject.UNDERSTANDING_SEARCH, # change this to WandbProject.DEMO_NOTEBOOKS!
	do_generate_dataset=False,
	dataset_verbose=True,
    dataset=DATASET,
)

2024-07-26 15:12:42 ERROR Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: miv. Use `wandb login --relogin` to force relogin


2024-07-26 15:12:46 INFO config ={'__format__': 'ConfigHolder(SerializableDataclass)', 'dataset_cfg': {'__format__': 'MazeDatasetConfig(SerializableDataclass)', 'name': 'demo_small', 'seq_len_min': 1, 'seq_len_max': 512, 'seed': 42, 'applied_filters': [], 'grid_n': 3, 'n_mazes': 100, 'maze_ctor': {'__name__': 'gen_dfs', '__module__': 'maze_dataset.generation.generators', '__doc__': ['generate a lattice maze using depth first search, iterative', '', '        # Arguments', '        - `grid_shape: Coord`: the shape of the grid', '        - `lattice_dim: int`: the dimension of the lattice', '          (default: `2`)', '        - `accessible_cells: int | float |None`: the number of accessible cells in the maze. If `None`, defaults to the total number of cells in the grid. if a float, asserts it is <= 1 and treats it as a proportion of **total cells**', '            (default: `None`)', '        - `max_tree_depth: int | float | None`: the maximum depth of the tree. If `None`, defaults to `2 *

F:\KNC\maze-transformer\maze_transformer\training\train_model.py:139: UserWarning:

dataset has different config than cfg.dataset_cfg, but the only difference is in applied_filters, so using passed dataset. This is due to fast dataset loading collecting generation metadata for performance reasons



2024-07-26 15:12:46 INFO finished getting training dataset with 100 samples
2024-07-26 15:12:46 INFO got validation dataset by splitting training dataset into 99 train and 1 validation samples
2024-07-26 15:12:46 INFO Loaded 99 sequences
2024-07-26 15:12:46 INFO Creating dataloader
2024-07-26 15:12:46 INFO finished dataloader, passing to train()
2024-07-26 15:12:46 INFO Initializing model
Moving model to device:  cpu
2024-07-26 15:12:46 INFO Initializing optimizer
2024-07-26 15:12:47 INFO will train for 7 batches, evals_enabled=True, with intervals: {'print_loss': inf, 'checkpoint': 3, 'eval_fast': 1, 'eval_slow': 3}
2024-07-26 15:12:47 INFO Starting training
2024-07-26 15:12:47 INFO Running evals: eval_fast


F:\KNC\maze-transformer\maze_transformer\evaluation\path_evals.py:98: RuntimeWarning:

fraction_connections_adjacent_lattice called on path of length less than 2, retuning NaN
prediction = array([[1, 2]])



2024-07-26 15:12:47 INFO Running evals: eval_slow
2024-07-26 15:12:47 INFO iteration 0/7: loss=3.198
2024-07-26 15:12:47 INFO Saving model checkpoint to ../data/multsrc_demo_small-g3-n100-a_dfs-h44636_nano-v1_test-v1_2024-07-26-15-12-39/checkpoints/model.iter_0.zanj
2024-07-26 15:12:47 INFO Running evals: eval_fast
2024-07-26 15:12:47 INFO Running evals: eval_fast
2024-07-26 15:12:47 INFO Running evals: eval_fast
2024-07-26 15:12:47 INFO Running evals: eval_slow
2024-07-26 15:12:47 INFO Saving model checkpoint to ../data/multsrc_demo_small-g3-n100-a_dfs-h44636_nano-v1_test-v1_2024-07-26-15-12-39/checkpoints/model.iter_3.zanj
2024-07-26 15:12:47 INFO Running evals: eval_fast
2024-07-26 15:12:47 INFO Running evals: eval_fast
2024-07-26 15:12:47 INFO Running evals: eval_fast
2024-07-26 15:12:47 INFO Running evals: eval_slow
2024-07-26 15:12:47 INFO Saving model checkpoint to ../data/multsrc_demo_small-g3-n100-a_dfs-h44636_nano-v1_test-v1_2024-07-26-15-12-39/checkpoints/model.iter_6.zanj
2